##### Importing Libs

In [1]:
import pandas as pd
import os
import sqlalchemy

##### Fetching file names

In [2]:
path_0 = os.path.dirname(os.getcwd()) + r"\raw_data\Streaming_History_Audio_"
path_n = ["2014-2016_0.json"] + [str(2016 + i) + "-" + str(2017 + i) + "_" + str(i + 1) + ".json" for i in range (8)]
path_7 = path_n[7]
path_7

'2022-2023_7.json'

##### Compiling everything in a DF

In [3]:
df_audio = pd.DataFrame() # (columns = pd.read_json(path_0 + path_n[0]).columns)

for path in path_n:
    df = pd.read_json(path_0 + path)
    print("Read: ", path, " / Shape: ", df.shape)
    df_audio = pd.concat([df_audio, df])

df_audio.reset_index(drop=True)
print("Final df shape: ", df_audio.shape) 

Read:  2014-2016_0.json  / Shape:  (15506, 21)
Read:  2016-2017_1.json  / Shape:  (17223, 21)
Read:  2017-2018_2.json  / Shape:  (16875, 21)
Read:  2018-2019_3.json  / Shape:  (16214, 21)
Read:  2019-2020_4.json  / Shape:  (16246, 21)
Read:  2020-2021_5.json  / Shape:  (16405, 21)
Read:  2021-2022_6.json  / Shape:  (16553, 21)
Read:  2022-2023_7.json  / Shape:  (16977, 21)
Read:  2023-2024_8.json  / Shape:  (15914, 21)
Final df shape:  (147913, 21)


##### Filtering data

In [4]:
df_audio = df_audio[df_audio['episode_show_name'].isna()] ## Removing podcasts
df_audio = df_audio[~df_audio['master_metadata_track_name'].isna()] ## Removing residual no info records

keep = ["ts",
        "ms_played",
        "conn_country",
        "master_metadata_track_name", 
        "master_metadata_album_artist_name",
        "master_metadata_album_album_name",
        "reason_start",
        "reason_end",
        "shuffle",
        "offline",
        "skipped"]

df_audio = df_audio[keep] ## Keeping only our columns of interest
df_audio.head(1)


,ts,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,reason_start,reason_end,shuffle,offline,skipped
0,2014-10-02T10:59:36Z,26563,BR,Candidate,Johnny Marr,Playland,appload,endplay,False,False,1.0


##### Creating db table

In [5]:
DB_PATH = os.path.dirname(os.getcwd()) + r"\spotify_ph.db"
engine = sqlalchemy.create_engine("sqlite:///" + DB_PATH)

engine.execute("DROP TABLE tb_spotify_ph")
df_audio.to_sql("tb_spotify_ph", engine, index = False)

C:\Users\jpgsa\AppData\Local\Temp\ipykernel_18864\3444234137.py:4: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute("DROP TABLE tb_spotify_ph")


146224